In [11]:
# 導入必要的庫
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# 讀取訓練資料集
df_train = pd.read_csv('trainv12.csv')  # 假設房價數據集存為 'trainv12.csv'

# 檢查缺失值
print("檢查缺失值：")
print(df_train.isnull().sum())
# 刪除無用的欄位（例如 'id', 'sale_yr' 等）
df_train.drop(columns=['id', 'sale_yr'], inplace=True)
# 如果有缺失值，用每列均值填補缺失值
df_train.fillna(df_train.mean(), inplace=True)  # 用每列均值填補缺失值（根據具體情況可調整）



# 處理 carprice 和 car_square 中的零值
# 將 carprice 中的零值替換為中位數
carprice_median = df_train['carprice'][df_train['carprice'] > 0].median()
df_train['carprice'] = df_train['carprice'].replace(0, carprice_median)

# 將 car_square 中的零值替換為中位數
car_square_median = df_train['car_square'][df_train['car_square'] > 0].median()
df_train['car_square'] = df_train['car_square'].replace(0, car_square_median)

# 分割特徵和目標變量
X_train = df_train.drop(columns=['price'])  # 假設 'price' 是目標變量
Y_train = df_train['price']

# 讀取驗證資料集
df_valid = pd.read_csv('validv4.csv')

# 保留驗證集的 'id' 欄位以便識別
df_valid.drop(columns=[ 'sale_yr'], inplace=True)

# 處理驗證資料中的缺失值和零值
# 填補缺失值
df_valid.fillna(df_valid.mean(), inplace=True)

# 將 carprice 中的零值替換為中位數
df_valid['carprice'] = df_valid['carprice'].replace(0, carprice_median)

# 將 car_square 中的零值替換為中位數
df_valid['car_square'] = df_valid['car_square'].replace(0, car_square_median)

# 分割驗證集的特徵和目標變量
X_valid = df_valid.drop(columns=['price'])  # 假設 'price' 是目標變量
Y_valid = df_valid['price']

# 讀取測試資料集
df_test = pd.read_csv('testv5.csv')

# 保留 'id' 欄位以便輸出結果
id_test = df_test['id']
df_test.drop(columns=[ 'id','sale_yr'], inplace=True)

# 處理測試資料中的缺失值和零值
# 填補缺失值
df_test.fillna(df_test.mean(), inplace=True)

# 將 carprice 中的零值替換為中位數
df_test['carprice'] = df_test['carprice'].replace(0, carprice_median)

# 將 car_square 中的零值替換為中位數
df_test['car_square'] = df_test['car_square'].replace(0, car_square_median)

# 分割測試集的特徵
X_test = df_test  # 測試集不包含 'price'，因為這是我們要預測的目標變量

# 標準化特徵
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# 建立 XGBRegressor 模型
model = XGBRegressor(
    learning_rate=0.1,  # 初始學習率，可以嘗試 0.01, 0.05, 0.1, 0.2
    n_estimators=200,    # 樹的數量，增大可以提高模型的擬合能力
    max_depth=5,         # 樹的最大深度，避免過擬合
    min_child_weight=10, # 葉節點最小樣本權重，增加可以提高泛化能力
    random_state=42      # 確保結果可重現
)

# 訓練模型
model.fit(X_train, Y_train)

# 使用驗證集進行預測並評估模型性能
Y_valid_pred = model.predict(X_valid)
mae_valid = mean_absolute_error(Y_valid, Y_valid_pred)
r2_valid = r2_score(Y_valid, Y_valid_pred)

print("驗證集模型性能：")
print(f"MAE (平均絕對誤差): {mae_valid:.3f}")
print(f"R^2 (決定係數): {r2_valid:.3f}")

# 預測測試集
Y_pred = model.predict(X_test)

# 將負的預測值替換為 0（以防萬一）
Y_pred = np.maximum(Y_pred, 0)

# 導出預測結果（只保留測試集的 id 和預測的 price）
result = pd.DataFrame({'id': id_test, 'price': Y_pred})
result.to_csv('predictions.csv', index=False)


檢查缺失值：
id                  0
price               0
sqft_above          0
sale_yr             0
grade               0
sqft_living         0
bedrooms            0
livingroom          0
bathrooms           0
price_per_square    0
car_square          0
carprice            0
dtype: int64
驗證集模型性能：
MAE (平均絕對誤差): 465524.818
R^2 (決定係數): 0.890
